[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aprendizaje-automatico-dc-uba-ar/material/blob/main/tp/01_aprendizaje_supervisado/tp01-enunciado.ipynb)

# Trabajo Práctico -  Aprendizaje supervisado
### Clasificación de expresiones genómicas

<span style="color: red;">**Fecha de entrega: 02 de Octubre del 2025 - hasta las 9:00hs.**
 
<span style="color: red;">**Fecha de entrega intermedia: 23 de Septiembre del 2025 - hasta las 9:00hs.**
</span>

## Introducción

En el mundo actual, distintas disciplinas científicas empiezan, cada vez más, a interactuar con el fin de potenciar sus descubrimientos. En este caso dos grupos de investigación de [CONICET](https://www.conicet.gov.ar/) se embarcan en la combinación entre biología y informática para abordar la detección temprana y el pronóstico preciso de enfermedades como el cáncer. Este proyecto combina las tecnologías de secuenciación de nueva generación ([_NGS_](https://es.wikipedia.org/wiki/Secuenciaci%C3%B3n_paralela_masiva), por sus siglas en inglés) con la potencia de la inteligencia artificial. El enfoque se centra en un dataset único que abarca mediciones de [_ARN_](https://es.wikipedia.org/wiki/ARN_mensajero) de 200 [_genes_](https://es.wikipedia.org/wiki/Gen), recopiladas de pacientes con lesiones [_pre-tumorales_](https://en.wikipedia.org/wiki/Hyperplasia). Este conjunto de datos se convierte en una valiosa fuente de información para entender cómo las células en estado de hiperplasia pueden evolucionar hacia [_tumores malignos_](https://en.wikipedia.org/wiki/Neoplasm), una transformación que ha desconcertado a la ciencia durante décadas.

La hiperplasia, es un fenómeno en el que las células experimentan un crecimiento anormal y descontrolado, es un punto de partida crucial en nuestro análisis. ¿Cómo y por qué algunas células que experimentan hiperplasia se convierten en células cancerosas, mientras que otras no? Esta pregunta es el corazón de nuestra investigación. Para responderla se realizo un estudio donde se obtuvieron muestras de distintos tipos de hiperplasias de pacientes con antecedentes familiares y lesiones pre tumorales. Este grupo de pacientes, o cohorte, fue monitoreado periodicamente durante los siguientes 5 años buscando indicios de neoplasias o nuevas hiperplasias más agresivas. Con las muestras obtenidas en este estudio se realizo un [_biobanco_](https://en.wikipedia.org/wiki/Biobank) con las mediciones que habitualmente se hacen en la construccion de este tipo de [_plataformas_](https://xena.ucsc.edu/). Cada muestra fue etiquetada como **_buen pronostico_**, si no hubo indicios de nuevas hiperplasias o similares; contrariamente se etiquetaron como de **_mal pronostico_** si hubo una recaida.

Este trabajo se concentra en un panel de genes, especificamente en la expresion de 200 genes que se creen tienen un papel crucial en la transformacion tumoral y su etiqueta correspondiente.

En concreto:

Tendrán un archivo `.csv` en donde se almacenan:
  - una matriz de datos `X` de $500$ filas en donde cada fila $x^{(i)}$ representa un vector de $200$ características de cada instancia. Es decir, $\textbf{x}^{(i)} = x_1^{(i)}, \dots, x_{200}^{(i)}$ con $i$ entre $1$ y $500$.
  - una columna llamada `target` que representa un vector de $500$ posiciones con dos posibles valores: `True` (ó 1, es decir, tiene buen pronostico) y `False` (ó 0, tiene mal pronostico).

Los datos están en esta [carpeta](https://github.com/aprendizaje-automatico-dc-uba-ar/material/tree/main/tp/01_aprendizaje_supervisado/datos).

Por otra parte, tendrán disponibles un conjunto de instancias sin etiquetas, que utilizaremos para comprobar la calidad de sus resultados (ver Ejercicio 4). 

**Recomendamos fuertemente leer primero todo el enunciado del trabajo antes de empezar a trabajar sobre el problema propuesto.**

---

### Sobre el informe

Para este trabajo deberán entregar, además del código de las pruebas y experimentos que realicen, un informe en el que deberan seleccionar, para cada apartado, sus resultados acompañado de un texto que explique, reflexione, justifique y conluya dicho contenido. 

Cada ejercicio indica el largo máximo del texto que se puede incluir. Los gráficos no están contados en dicho espacio. 
Cada gráfico incluido debe contar con:
  
  - nombres de los ejes,
  - título,
  - leyenda autocontenida,
  - debe ser referenciado desde el texto, ya que su inclusión se da porque aporta a la discusión del trabajo.

**El informe no puede superar un máximo de 8 carillas (contando gráficos) o 4 hojas más carátula.** No se corregirán trabajos que no cumplan con esta consigna.

---

### Funciones Auxiliares

In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import precision_recall_curve, roc_curve, auc

def accuracy(y_predicted: np.ndarray, y_real: np.ndarray) -> float:
    TP_TN = sum([y_i == y_j for (y_i, y_j) in zip(y_predicted, y_real)]) 
    P_N = len(y_real)
    return TP_TN / P_N

def get_df_metricas(predictions, scores, true_labels):
    k = len(predictions)

    rows = []
    y_true_all = []
    y_pred_all = []
    scores_all = []

    for i in range(k):
        y_train, y_test = true_labels[i]
        y_pred_train, y_pred_test = predictions[i]
        y_score_train, y_score_test = scores[i]
        
        y_true_all.append(y_test)
        y_pred_all.append(y_pred_test)
        scores_all.append(y_score_test)

        # AUPRC training
        prec_tr, rec_tr, _ = precision_recall_curve(y_train, y_score_train)
        auprc_train = auc(rec_tr, prec_tr)

        # AUPRC validación
        prec_te, rec_te, _ = precision_recall_curve(y_test, y_score_test)
        auprc_val = auc(rec_te, prec_te)

        # AUC ROC training
        fpr_tr, tpr_tr, _ = roc_curve(y_train, y_score_train)
        aucroc_train = auc(fpr_tr, tpr_tr)

        # AUC ROC validación
        fpr_te, tpr_te, _ = roc_curve(y_test, y_score_test)
        aucroc_val = auc(fpr_te, tpr_te)

        row = {
            "Permutación": i + 1,
            "Accuracy (training)": accuracy(y_pred_train, y_train),
            "Accuracy (validación)": accuracy(y_pred_test, y_test),
            "AUPRC (training)": auprc_train,
            "AUPRC (validación)": auprc_val,
            "AUC ROC (training)": aucroc_train,
            "AUC ROC (validación)": aucroc_val,
        }

        rows.append(row)

    df = pd.DataFrame(rows)
    promedio = df.mean(numeric_only=True)
    promedio["Permutación"] = "Promedios"
    df = pd.concat([df, pd.DataFrame([promedio])], ignore_index=True)

    y_true_all = np.concatenate(y_true_all)
    y_pred_all = np.concatenate(y_pred_all)
    scores_all = np.concatenate(scores_all)

    prec_g, rec_g, _ = precision_recall_curve(y_true_all, scores_all)
    auprc_global = auc(rec_g, prec_g)

    fpr_g, tpr_g, _ = roc_curve(y_true_all, scores_all)
    aucroc_global = auc(fpr_g, tpr_g)

    fila_global = {
        "Permutación": "Global",
        "Accuracy (training)": "(NO)",
        "Accuracy (validación)": accuracy(y_pred_all, y_true_all),
        "AUPRC (training)": "(NO)",
        "AUPRC (validación)": auprc_global,
        "AUC ROC (training)": "(NO)",
        "AUC ROC (validación)": aucroc_global,
    }

    df = pd.concat([df, pd.DataFrame([fila_global])], ignore_index=True)

    return df

In [2]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import ParameterGrid
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=DeprecationWarning)

df = pd.read_csv('data.csv')
df.head()

,uAro,vIlg,VtZP,qTUs,bgVs,PdRG,CqGH,bfBD,ZtLG,SNCN,...,hnYp,cWwQ,GkqC,VHPu,KKxk,TNkz,GkcC,KSYM,YpTu,target
0,1.284843,0.676813,-0.445511,-0.025185,-4.546595,0.370719,0.104766,-0.365206,1.155584,0.791519,...,2.052021,-0.256875,-0.961930,4.731984,21.095664,-0.738587,0.728201,1.346252,0.659334,1
1,-0.027713,1.020162,-1.665049,-1.052946,3.144244,2.104813,0.603191,0.180639,1.458030,-0.014031,...,0.891111,-1.328636,-0.848692,-3.989681,7.741832,-0.768312,0.160028,-1.028335,-1.733235,1
2,-0.160656,-1.011420,-0.216399,0.797341,5.672563,7.822798,0.806452,0.386419,-0.854304,0.003736,...,1.411142,2.481312,0.492739,-3.132938,-22.681919,-1.066736,-0.041351,-0.311166,-1.299748,0
3,1.128357,0.341273,-1.669474,0.767591,-4.610520,-8.788584,2.272761,-0.762237,-0.245964,-0.101648,...,1.418474,-0.555290,-0.400534,0.149306,-8.262423,-4.142330,1.256389,-1.252168,-1.597531,0
4,0.573025,1.278193,0.112418,-1.431578,0.589556,1.913852,0.021793,0.533563,0.321761,-0.108064,...,-0.487612,-0.818391,-0.158147,-4.996113,-18.901603,-1.276128,0.318565,-0.592519,1.788781,0


## Ejercicio 1 

### Separación de datos

Contarán con una cantidad limitada de datos, por lo cual es importante tomar una buena decisión en el momento de empezar a utilizarlos. 

Evaluar y justificar cómo separarán sus datos para desarrollo y para evaluación. ¿Qué consideraciones tuvieron en cuenta para realizar esta división?

**Importante**: en este punto no está permitido dividir la base de datos utilizando la función `train_test_split` de sklearn. Deben decidir e implementar la separación.

### Consideraciones

Mantener la distribución de la variable objetivo, 'target'. Esto es importante porque suponemos que la distribución de los datos se condice con la realidad de los pronósticos.

In [3]:
df_target = df[df['target'] == 1]
print("Proporción de buenos pronósticos", str(len(df_target) / len(df) * 100), "%")

Proporción de buenos pronósticos 29.4 %


In [38]:
import math

def test_train_split(frac, seed=42):

    np.random.seed(seed) 

    sample_positive = df[df["target"] == 1].sample(frac=frac,random_state=seed).index
    sample_negative = df[df["target"] == 0].sample(frac=frac,random_state=seed).index

    train = df.loc[np.append(sample_positive, sample_negative)]    
    test = df.drop(train.index)
    
    # mezclamos porque antes concatenamos positivos con negativos
    train = train.sample(frac=1, random_state=seed).reset_index(drop=True)
    
    return train, test

desarrollo, evaluacion = test_train_split(frac=0.9)

## Ejercicio 2


### Construcción de modelos

Para este punto, la tarea consiste en construir y evaluar modelos de tipo **árbol de decisión**. Además, obtener una **estimación realista de la performance** de los mismos.

1. Entrenar un árbol de decisión con altura máxima 3 y el resto de los hiperparámetros en default.

1. Estimar la performance del modelo utilizando _K-fold cross validation_ con `K=5`, con las métricas _Accuracy_, _Area Under the Precision-Recall Curve (AUPRC)_, y _Area Under the Receiver Operating Characteristic Curve (AUCROC)_. 

   En esta oportunidad se va a pedir además de calcular las métricas para cada fold por separado y su promedio, que hagan el cálculo del score global (como vimos en clase), sólo para los folds de validación.
   
   Reportar el resultado en una tabla similar a:

      <table>
      <thead>
      <tr>
      <th align="center">Permutación</th>
      <th>Accuracy (training)</th>
      <th>Accuracy (validación)</th>
      <th>AUPRC (training)</th>
      <th>AUPRC (validación)</th>
      <th>AUC ROC (training)</th>
      <th>AUC ROC (validación)</th>
      </tr>
      </thead>
      <tbody>
      <tr>
      <td align="center">1</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      </tr>
      <tr>
      <td align="center">2</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      </tr>
      <tr>
      <td align="center">3</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      </tr>
      <tr>
      <td align="center">4</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      </tr>
      <tr>
      <td align="center">5</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      </tr>
      <tr>
      <td align="center">Promedios</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      </tr>
      <td align="center">Global</td>
      <td>(NO) </td>
      <td></td>
      <td>(NO) </td>
      <td></td>
      <td>(NO) </td>
      <td></td>
      </tr>
      </tbody>
      </table>    
  
   **Importante**: de acá en más sólamente utilizaremos el score promedio cuando hagamos _K-fold cross-validation_.
 
1. Explorar las siguientes combinaciones de parámetros para  árboles de decisión (siguiendo con $k-fold$ con $k=5$) utilizando [ParameterGrid](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ParameterGrid.html) de _scikit learn_. No está permitido utilizar `GridSearchCV` en este ejercicio.

   <table>
   <thead>
   <tr>
   <th align="center">Altura máxima</th>
   <th align="center">Criterio de corte</th>
   <th>Accuracy (training)</th>
   <th>Accuracy (validación)</th>
   </tr>
   </thead>
   <tbody><tr>
   <td align="center">3</td>
   <td align="center">Gini</td>
   <td></td>
   <td></td>
   </tr>
   <tr>
   <td align="center">5</td>
   <td align="center">Gini</td>
   <td></td>
   <td></td>
   </tr>
   <tr>
   <td align="center">Infinito</td>
   <td align="center">Gini</td>
   <td></td>
   <td></td>
   </tr>
   <tr>
   <td align="center">3</td>
   <td align="center">Entropía</td>
   <td></td>
   <td></td>
   </tr>
   <tr>
   <td align="center">5</td>
   <td align="center">Entropía</td>
   <td></td>
   <td></td>
   </tr>
   <tr>
   <td align="center">Infinito</td>
   <td align="center">Entropía</td>
   <td></td>
   <td></td>
   </tr>
   </tbody></table>

1. ¿Qué conclusiones se pueden sacar de estas tablas?  

In [39]:
def _scores(modelo, X):
    if hasattr(modelo, "predict_proba"):
        return modelo.predict_proba(X)[:, 1]
    if hasattr(modelo, "decision_function"):
        return modelo.decision_function(X)
    return modelo.predict(X)

def k_fold(df, model, params, k=5, verbose=False, seed=42):
    np.random.seed(seed) 

    # Toma un dataframe, el modelo 
    #los parametros del modelo y devuelve las predicciones con sus score y la clase real  
    predictions = []
    true_labels = []
    scores = []
    
    df_0 = df[df['target'] == 0].sample(frac=1, random_state=seed)
    df_1 = df[df['target'] == 1].sample(frac=1, random_state=seed)
    
    folds_0 = np.array_split(df_0, k)
    folds_1 = np.array_split(df_1, k)
    
    for k_fold in range(k):
        if verbose:
            print(f"Fold {k_fold + 1}")
    
        test_0 = folds_0[k_fold]
        test_1 = folds_1[k_fold]
        test = pd.concat([test_0, test_1])
        
        train_0 = pd.concat(folds_0[:k_fold] + folds_0[k_fold+1:])
        train_1 = pd.concat(folds_1[:k_fold] + folds_1[k_fold+1:])
        train = pd.concat([train_0, train_1])
            
        modelo = model(**params)

        x_train = train.iloc[:, 0:200].to_numpy()
        y_train = train['target'].to_numpy()

        if np.unique(y_train).size < 2:
            raise Exception("Train Fold con 1 sola clase.")

        x_test = test.iloc[:, 0:200].to_numpy()
        y_test = test['target'].to_numpy()
    
        if verbose:
            print("Train positivos:", sum(y_train == 1), "negativos:", sum(y_train == 0))
            print("Test positivos:", sum(y_test == 1), "negativos:", sum(y_test == 0))    
            
        modelo.fit(x_train, y_train)
    
        y_predict_test = modelo.predict(x_test)
        y_predict_train = modelo.predict(x_train)

        y_score_test  = _scores(modelo, x_test)
        y_score_train = _scores(modelo, x_train)
        
        predictions.append((y_predict_train, y_predict_test))
        true_labels.append((y_train, y_test))
        scores.append((y_score_train, y_score_test))       

    return predictions, scores, true_labels

In [40]:
predictions, scores, true_labels = k_fold(desarrollo, DecisionTreeClassifier, {"max_depth":3}, verbose=True)
get_df_metricas(predictions, scores, true_labels)

Fold 1
Train positivos: 105 negativos: 254
Test positivos: 27 negativos: 64
Fold 2
Train positivos: 105 negativos: 254
Test positivos: 27 negativos: 64
Fold 3
Train positivos: 106 negativos: 254
Test positivos: 26 negativos: 64
Fold 4
Train positivos: 106 negativos: 255
Test positivos: 26 negativos: 63
Fold 5
Train positivos: 106 negativos: 255
Test positivos: 26 negativos: 63


,Permutación,Accuracy (training),Accuracy (validación),AUPRC (training),AUPRC (validación),AUC ROC (training),AUC ROC (validación)
0,1,0.846797,0.648352,0.785425,0.307453,0.862767,0.541956
1,2,0.793872,0.692308,0.749168,0.463050,0.858455,0.680845
2,3,0.841667,0.655556,0.726518,0.332054,0.855259,0.619291
3,4,0.855956,0.719101,0.802004,0.399875,0.85875,0.692613
4,5,0.808864,0.617978,0.772121,0.479947,0.879449,0.669109
5,Promedios,0.829431,0.666659,0.767047,0.396476,0.862936,0.640763
6,Global,(NO),0.666667,(NO),0.408319,(NO),0.647584


In [41]:
def get_promedios(df_metricas):
    return df_metricas[df_metricas["Permutación"] == "Promedios"].iloc[0]    

grilla = {
    "max_depth": [3, 5, None],
    "criterion":["gini", "entropy"],
}

resultados = []
for params in ParameterGrid(grilla):
    predictions, scores, true_labels = k_fold(desarrollo, DecisionTreeClassifier, params)
    df_metricas = get_df_metricas(predictions, scores, true_labels)
    
    fila_promedios = get_promedios(df_metricas)
    fila = {
        "Altura máxima": params["max_depth"] if params["max_depth"] is not None else "infinito",
        "Criterio de corte": params["criterion"],
        "Accuracy (training)": fila_promedios["Accuracy (training)"],
        "Accuracy (validación)": fila_promedios["Accuracy (validación)"],
    }
    resultados.append(fila)

resultados = pd.DataFrame(resultados)
resultados

,Altura máxima,Criterio de corte,Accuracy (training),Accuracy (validación)
0,3,gini,0.829431,0.666659
1,5,gini,0.942202,0.651199
2,infinito,gini,1.000000,0.657472
3,3,entropy,0.808320,0.691031
4,5,entropy,0.912757,0.673351
5,infinito,entropy,1.000000,0.646435


## Ejercicio 3

### Comparación de algoritmos 

Se pide explorar distintas combinaciones de algoritmos de aprendizaje con diferentes configuraciones con el objetivo de **encontrar el mejor modelo** de cada familia de buscar la performance óptima. Para este ejercicio realizar una experimentación utilizando [`RandomizedSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html). Como métrica de performance usar AUCROC resultante de 5-fold cross-validation. 

Hiperparámetros_: Revisar la documentación de cada uno de los hiperparámetros para entender qué posibles hiperparámetros impacten de manera positiva en la construcción del algoritmo.

Documentación extra sobre [`Tuning hyper-parameters`](https://scikit-learn.org/stable/modules/grid_search.html), leer hasta 3.2.2.

1. Algoritmos a probar: 
  - Árboles de decisión. Mínimo 4 hiperparámetros.
  - KNN (k-vecinos más cercanos). Mínimo 3 hiperparámetros.
  - SVM (Support vector machine). Mínimo 2 hiperparámetros.

Detallar los hiperparámetros elegidos para cada algoritmo y explicar la razón del espacio de búsqueda considerado para cada uno de estos, ¿cuántas iteraciones usaron?. A su vez, reportar la performance asociada de aquellos que consideren relevantes (al menos la mejor combinación para cada algoritmo). 

2. Compare los resultados obtenidos en el ejercicio anterior con los siguientes modelos con sus hiperparámetros default. 

  - LDA (Linear discriminant analysis)
  - Naïve Bayes

¿Qué resultados obtuvo? ¿Qué hiperparámetros podrían ser relevantes explorar en estos modelos? ¿Por qué?

3. ¿Cuál fue el mejor modelo y con qué configuración? Explicar por qué creería que dio mejor (recordando qué hace cada algoritmo y con qué tipo de datos están trabajando).

In [42]:
#EJ 3
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint, uniform
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV, RepeatedStratifiedKFold

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=20, random_state=42)

param_arbol = {
    "criterion": ["gini", "entropy"],          # categorías → lista normal
    "max_depth": randint(3, 20),               # enteros entre 3 y 19
    "max_features": uniform(0.1, 0.9),         # flotantes entre 0.1 y 1.0 (10%–100% de features)
    "min_samples_leaf": randint(1, 20),        # enteros entre 1 y 19
} 

param_KNN = {
    "p":[1,2,np.inf],
    "n_neighbors":randint(3, 20),
    "weights":["uniform", "distance"] #pesa los votos segun inversamente proporcional a las distancia
} 


arbol=DecisionTreeClassifier(random_state=42)
knn=KNeighborsClassifier()

random_search_arbol = RandomizedSearchCV(
    estimator=arbol,
    param_distributions=param_arbol,
    n_iter=20,          # cantidad de combinaciones a probar
    cv=cv,               # cross-validation
    scoring="roc_auc",
    random_state=42
)

random_search_knn = RandomizedSearchCV(
    estimator=knn,
    param_distributions=param_KNN,
    n_iter=20,          # cantidad de combinaciones a probar
    cv=cv,               # cross-validation
    scoring="roc_auc",
    random_state=42
)

x_train = desarrollo.iloc[:,0:200].to_numpy()
y_train = desarrollo['target'].to_numpy()

random_search_arbol.fit(x_train,y_train)
random_search_knn.fit(x_train,y_train)

,estimator,KNeighborsClassifier()
,param_distributions,"{'n_neighbors': <scipy.stats....t 0x1562e3d00>, 'p': [1, 2, ...], 'weights': ['uniform', 'distance']}"
,n_iter,20
,scoring,'roc_auc'
,n_jobs,None
,refit,True
,cv,RepeatedStrat...ndom_state=42)
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [43]:
result_arbol=pd.DataFrame(random_search_arbol.cv_results_)
result_arbol=result_arbol.sort_values(by="mean_test_score",ascending=False)
result_arbol["criterion"] = result_arbol["params"].apply(lambda x: x["criterion"])
result_arbol["maxima profundidad"] = result_arbol["params"].apply(lambda x: x["max_depth"])
result_arbol["max_features"] = result_arbol["params"].apply(lambda x: x["max_features"])
result_arbol["min_samples_leaf"] = result_arbol["params"].apply(lambda x: x["min_samples_leaf"])
result_arbol=result_arbol[["criterion","maxima profundidad","max_features","min_samples_leaf","mean_test_score","std_test_score"]]
result_arbol.head(5)

,criterion,maxima profundidad,max_features,min_samples_leaf,mean_test_score,std_test_score
6,entropy,18,0.309494,15,0.671800,0.060169
9,entropy,6,0.947982,14,0.671279,0.058239
18,entropy,17,0.845864,13,0.668840,0.060926
17,gini,10,0.860080,13,0.661785,0.076947
11,entropy,17,0.714937,12,0.654798,0.060589


In [44]:
result_knn=pd.DataFrame(random_search_knn.cv_results_)
result_knn=result_knn.sort_values(by="mean_test_score",ascending=False)
result_knn["n_neighbors"]=result_knn["params"].apply(lambda x: x["n_neighbors"])
result_knn["p"]=result_knn["params"].apply(lambda x: x["p"])
result_knn["weights"]=result_knn["params"].apply(lambda x: x["weights"])
result_knn=result_knn[["n_neighbors","p","weights","mean_test_score","std_test_score"]]
result_knn.head(5)

,n_neighbors,p,weights,mean_test_score,std_test_score
18,9,1.0,distance,0.882648,0.038874
8,14,1.0,uniform,0.881182,0.038242
1,13,1.0,uniform,0.879503,0.038895
0,9,1.0,uniform,0.870035,0.039747
13,9,1.0,uniform,0.870035,0.039747


In [45]:
param_SVM={
    "C":randint(1,300),
    "kernel":["rbf", "poly", "sigmoid"]
} 

svc=SVC(random_state=42)

random_search_svc = RandomizedSearchCV(
    estimator=svc,
    param_distributions=param_SVM,
    n_iter=20,          # cantidad de combinaciones a probar
    cv=cv,               # cross-validation
    scoring="roc_auc",
    random_state=42
)

random_search_svc.fit(x_train,y_train)
result_svc=pd.DataFrame(random_search_svc.cv_results_)
result_svc=result_svc.sort_values(by="mean_test_score",ascending=False)
result_svc["C"]=result_svc["params"].apply(lambda x: x["C"])
result_svc["kernel"]=result_svc["params"].apply(lambda x: x["kernel"])
result_svc=result_svc[["C","kernel","mean_test_score","std_test_score"]]
result_svc.head(5)

,C,kernel,mean_test_score,std_test_score
0,103,rbf,0.906976,0.033098
11,192,rbf,0.906976,0.033098
2,72,rbf,0.906976,0.033098
14,236,rbf,0.906976,0.033098
13,22,rbf,0.906976,0.033098


In [46]:
random_search_gauss = RandomizedSearchCV(
    estimator=GaussianNB(),
    param_distributions={},
    n_iter=1,          # cantidad de combinaciones a probar
    cv=cv,               # cross-validation
    scoring="roc_auc",
    random_state=42
)

random_search_gauss.fit(x_train,y_train)

result_gauss=pd.DataFrame(random_search_gauss.cv_results_)
result_gauss=result_gauss.sort_values(by="mean_test_score",ascending=False)
result_gauss=result_gauss[["mean_test_score","std_test_score"]]
result_gauss.head(5)

,mean_test_score,std_test_score
0,0.809015,0.042554


In [47]:
random_search_lda = RandomizedSearchCV(
    estimator=LDA(),
    param_distributions={},
    n_iter=1,          # cantidad de combinaciones a probar
    cv=cv,               # cross-validation
    scoring="roc_auc",
    random_state=42
)

random_search_lda.fit(x_train,y_train)

result_lda=pd.DataFrame(random_search_lda.cv_results_)
result_lda=result_lda.sort_values(by="mean_test_score",ascending=False)
result_lda=result_lda[["mean_test_score","std_test_score"]]
result_lda.head(5)

,mean_test_score,std_test_score
0,0.755983,0.058729


## Ejercicio 4: 
### Evaluación de performance

- La entrega del trabajo estará acompañada de una evaluación en la cual deberán poner a prueba su mejor modelo y sobre todo, su capacidad para estimar sus resultados. 

- Su tarea será estimar la performance (AUCROC) que tendrá su mejor modelo en datos de evaluación (X_held_out). 

- Para ello, deberán predecir las **probabilidades** (probabilidad de la clase positiva) de las distintas instancias con su modelo, enviarnos dichas probabilidades junto a una estimación con 4 decimales de cuál será el AUCROC resultante y calcularemos el resultado real. Consideraremos que el **mejor modelo será el que se encuentre más cerca del valor real que calcularemos luego de la entrega**.

- Recomendamos no perder de vista esta evaluación/competencia durante el desarrollo del TP, sobretodo en el momento de separar los datos en los primeros puntos. 

- Para que podamos evaluar la performance, junto con la entrega del informe, deberán enviar un archivo con el numero de grupo con dos digitos en formato csv con la columna `output` y el valor obtenido con 4 decimales (se subirá un ejemplo cuando se publiquen los datos de la competencia) y un valor esperado de AUCROC: `GG_y_pred_held_out_AUCROC`. 

    - Ej.: el grupo tres cree que obtuvo un valor de 0.7321 de AUCROC deberá submitear un archivo llamado: `03_y_pred_held_out_7321.csv`.

- Los datos podrán encontrarlos en este [link](https://github.com/aprendizaje-automatico-dc-uba-ar/material/tree/main/tp/01_aprendizaje_supervisado/datos).

- Las decisiones de este punto pueden desarrollarse hasta en una carilla, aunque con media debería alcanzar.


In [48]:
from sklearn.metrics import roc_auc_score
held_out  = pd.read_csv('X_held_out.csv') 

def split_X_y(df: pd.DataFrame):
  cols = list(df.columns)
  y = df['target'].to_numpy() if 'target' in cols else None
  X = df.drop(columns=['target']) if 'target' in cols else df.copy()
  return X, y

In [49]:
# KNN
mejor_modelo = random_search_knn.best_estimator_
mejor_modelo.set_params()

X_dev, y_dev = split_X_y(desarrollo)
X_val, y_val = split_X_y(evaluacion)

mejor_modelo.fit(X_dev, y_dev)
y_val_scores = mejor_modelo.predict(X_val)

auc_val = roc_auc_score(y_val, y_val_scores)
print(f"AUC-ROC estimado (validación 10%): {auc_val:.4f}")

mejor_modelo.fit(X_dev, y_dev)

X_held = held_out.copy()
y_scores = mejor_modelo.predict(X_held)

df_submit = pd.DataFrame({'output': y_scores})
df_submit['output'] = df_submit['output'].apply(lambda x: round(x, 4))

AUC-ROC estimado (validación 10%): 0.5190


In [50]:
# SVC
mejor_modelo = random_search_svc.best_estimator_
mejor_modelo.set_params(random_state=42)

X_dev, y_dev = split_X_y(desarrollo)
X_val, y_val = split_X_y(evaluacion)

mejor_modelo.fit(X_dev, y_dev)
y_val_scores = mejor_modelo.decision_function(X_val)

auc_val = roc_auc_score(y_val, y_val_scores)
print(f"AUC-ROC estimado (validación 10%): {auc_val:.4f}")

X_held = held_out.copy()
y_scores = mejor_modelo.decision_function(X_held)

df_submit = pd.DataFrame({'output': y_scores})
df_submit['output'] = df_submit['output'].apply(lambda x: round(x, 4))

AUC-ROC estimado (validación 10%): 0.8990


In [64]:
# Bootstrap the predictions:
rng = np.random.RandomState(seed=12345)
idx = np.arange(y_val.shape[0])

test_accuracies = []

for i in range(300):
    pred_idx = rng.choice(idx, size=idx.shape[0], replace=True)    
    auc_val_boot = roc_auc_score(y_val[pred_idx], y_val_scores[pred_idx])
    test_accuracies.append(auc_val_boot)

bootstrap_test_mean = np.mean(test_accuracies)
print(f"AUC-ROC bootstrap estimado (validación 10%): {bootstrap_test_mean:.4f}")

grupo = "03"
auc_roc = 8990 
fname = f"{grupo}_y_pred_held_out_{bootstrap_test_mean:.4f}.csv"

df_submit.to_csv(fname, index=False)
print("Filas:", len(df_submit))
df_submit.head()

AUC-ROC bootstrap estimado (validación 10%): 0.9038
Filas: 500


,output
0,-1.3010
1,-0.7254
2,-0.6640
3,-0.4337
4,-1.4398


In [52]:
ci_lower = np.percentile(test_accuracies, 2.5)
ci_upper = np.percentile(test_accuracies, 97.5)

print(ci_lower, ci_upper)

0.7887390873015874 0.9905448951048951


In [61]:
# Veamos que features son las mas importantes
from sklearn.inspection import permutation_importance

result = permutation_importance(mejor_modelo, X_val, y_val, n_repeats=20, random_state=42, n_jobs=-1)

importances = result.importances_mean
sorted_idx = importances.argsort()[::-1]

# Mostrar top 20 features más importantes
top_n = 5
for i in range(top_n):
    print(f"Feature {sorted_idx[i]}: importance = {importances[sorted_idx[i]]:.4f}")

Feature 27: importance = 0.0670
Feature 83: importance = 0.0540
Feature 142: importance = 0.0470
Feature 176: importance = 0.0450
Feature 116: importance = 0.0440


In [54]:
# Calculamos la media y la varianza del top 5 de las features mas importantes de nuestro set de validacion y comparamos con 
# el de X_held_out para ver si la distribución tiene al menos estadísticos similares:

# Trainning
top5_idx = sorted_idx[:5]

print("--- TRAINNING")
variances = X_dev.iloc[:, top5_idx].var(axis=0)  # muestral por default en pandas
means = X_dev.iloc[:, top5_idx].mean(axis=0)  # muestral por default en pandas

for col, mean, var in zip(X_val.columns[top5_idx], means, variances):
    print(f"Feature {col}: mean = {mean:4f} & var = {var:.4f}")

print("--- VALIDACION")
variances = X_val.iloc[:, top5_idx].var(axis=0)  # muestral por default en pandas
means = X_val.iloc[:, top5_idx].mean(axis=0)  # muestral por default en pandas

for col, mean, var in zip(X_val.columns[top5_idx], means, variances):
    print(f"Feature {col}: mean = {mean:4f} & var = {var:.4f}")

print("--- HELD OUT")
variances = X_held.iloc[:, top5_idx].var(axis=0)  # muestral por default en pandas
means = X_held.iloc[:, top5_idx].mean(axis=0)  # muestral por default en pandas

for col, mean, var in zip(X_val.columns[top5_idx], means, variances):
    print(f"Feature {col}: mean = {mean:4f} & var = {var:.4f}")

--- TRAINNING
Feature Npsq: mean = 1.775825 & var = 193.5543
Feature BAjk: mean = -0.313504 & var = 278.9872
Feature vNma: mean = 1.118302 & var = 203.7573
Feature UXNV: mean = -0.744320 & var = 13.2710
Feature oanJ: mean = 0.135432 & var = 16.1919
--- VALIDACION
Feature Npsq: mean = 2.927779 & var = 241.4056
Feature BAjk: mean = 3.329533 & var = 305.4371
Feature vNma: mean = 2.505020 & var = 140.1498
Feature UXNV: mean = -0.708319 & var = 16.3015
Feature oanJ: mean = -0.720608 & var = 15.5346
--- HELD OUT
Feature Npsq: mean = 0.367800 & var = 181.5105
Feature BAjk: mean = -1.641785 & var = 260.2267
Feature vNma: mean = 0.801940 & var = 192.1371
Feature UXNV: mean = -0.566916 & var = 13.8255
Feature oanJ: mean = 0.493406 & var = 16.3012


## Ejercicio 5: 
### Conclusiones

Escribir como mínimo en un párrafo, una conclusión del trabajo realizado, incluyendo problemas encontrados y 
aspectos no incluidos en el enunciado que hayan sido abordadas durante el desarrollo.

---
## Entregables
- Contarán con un esqueleto en formato Jupyter Notebook en donde podrán intercalar celdas para reportar y responder a los ítems de cada ejercicio. 
- Los entregrables serán
    - Un informe en formato .pdf (**digital**) que responda a los ítems de este enunciado respetando la cantidad de espacio máximo por cada ítem. Nombrarlo siguiendo el formato `GG_Nombre_de_grupo`
    - Adjuntar el notebook final en formatos .pdf e .ipynb. Es necesario que los resultados puedan reproducirse al ejecutar todas las celdas en orden (verificarlo haceindo: Kernel -> Restart and Run All). 
    - Las predicciones del *held out* del punto 4 en formato csv.
- Habrá una entrega intermedia obligatoria que deberán hacer antes de la fecha indicada en el campus. Para esta entrega deberán enviar el código que resuelve los primeros 3 ejercicios. 
- La **fecha** y **hora límite** de entrega está determinada en el campus de la materia.
- El trabajo deberá elaborarse en grupos de 4 personas.
- Se podrán pedir pruebas de integridad y autoría; es decir, verificar que la salida solicitada es fruto del modelo presentado y que el modelo fue construido según lo requerido en este enunciado.
- La evaluación será grupal y se basará en la calidad del informe (presentación, claridad, prolijidad); la originalidad, practicidad y coherencia técnica de la solución; la corrección y solidez de las pruebas realizadas.


### Importante: sobre el uso de ChatGPT y grandes modelos de lenguaje

En este trabajo no estará explícitamente prohibido pero si fuertemente desaconsejado, consideramos a este trabajo práctico una importante herramienta de aprendizaje donde el uso de GPT puede ser perjudicial. En caso de usarlo se pide aclararlo en el informe y especificar cómo y en donde se utilizó. Así como expresar su opinión sobre la respuesta generada por el modelo pudiendo estar a favor o en contra de lo propuesto por este. Pueden adjuntar el link a la conversación con el modelo.

**Nota**: Agradecemos a [Martín García Sola](https://ar.linkedin.com/in/martin-e-garcia-sola) por la asistencia biológica en la confección de este Trabajo Práctico.